In [1]:
# confirm working in virtual environment
import sys
display(sys.version)
display(sys.executable)

'3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]'

'c:\\dev\\python_visualizations\\venv\\scripts\\python.exe'

In [2]:
# imports
import os
import re
import zipfile

import pandas as pd
import openpyxl
import missingno as msno

# notebook configuration
from IPython.display import display
pd.options.display.max_columns = None  # show all columns

# visualization
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt


In [3]:
# get list of raw csv and xlsx data files
# csv is data
# xlsx is data dictionary - multiple tabs
data_dir_raw = os.path.join("..", "data", "raw_v2")
data_files_raw = [os.path.join(data_dir_raw, file) for file in os.listdir(data_dir_raw) if file.endswith((".csv", ".xlsx"))]
data_files_raw

['..\\data\\raw_v2\\adm2017.csv',
 '..\\data\\raw_v2\\adm2017.xlsx',
 '..\\data\\raw_v2\\hd2017.csv',
 '..\\data\\raw_v2\\hd2017.xlsx',
 '..\\data\\raw_v2\\ic2017.csv',
 '..\\data\\raw_v2\\ic2017.xlsx',
 '..\\data\\raw_v2\\ic2017_ay.csv',
 '..\\data\\raw_v2\\ic2017_ay.xlsx',
 '..\\data\\raw_v2\\Most-Recent-Cohorts-Treasury-Elements.csv',
 '..\\data\\raw_v2\\US-News-Rankings-Liberal-Arts-Colleges-Through-2020.xlsx',
 '..\\data\\raw_v2\\US-News-Rankings-Universities-Through-2020.xlsx',
 '..\\data\\raw_v2\\~$adm2017.xlsx']

#### not working -  permissions issue 
for file in data_dir_raw:
    with zipfile.ZipFile(file,"r") as zip_ref:
        print("extracting ", file)
        zip_ref.extractall(file)

In [4]:
# import core college data - name, IPEDS ID number, address, etc.
college_core_data = os.path.join(data_dir_raw, "hd2017.csv")
df_colleges = pd.read_csv(college_core_data, encoding = "ISO-8859-1")  # encoding required, "utf-8" encoding fails
df_colleges.head(5)

,UNITID,INSTNM,IALIAS,ADDR,CITY,STABBR,ZIP,FIPS,OBEREG,CHFNM,CHFTITLE,GENTELE,EIN,DUNS,OPEID,OPEFLAG,WEBADDR,ADMINURL,FAIDURL,APPLURL,NPRICURL,VETURL,ATHURL,DISAURL,SECTOR,ICLEVEL,CONTROL,HLOFFER,UGOFFER,GROFFER,HDEGOFR1,DEGGRANT,HBCU,HOSPITAL,MEDICAL,TRIBAL,LOCALE,OPENPUBL,ACT,NEWID,DEATHYR,CLOSEDAT,CYACTIVE,POSTSEC,PSEFLAG,PSET4FLG,RPTMTH,INSTCAT,C15BASIC,C15IPUG,C15IPGRD,C15UGPRF,C15ENPRF,C15SZSET,CCBASIC,CARNEGIE,LANDGRNT,INSTSIZE,F1SYSTYP,F1SYSNAM,F1SYSCOD,CBSA,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG
0,100654,Alabama A & M University,AAMU,4900 Meridian Street,Normal,AL,35762,1,5,"Dr. Andrew Hugine, Jr.",President,2563725000,636001109,197216455,00100200,1,www.aamu.edu/,www.aamu.edu/Admissions/Pages/default.aspx,www.aamu.edu/Admissions/fincialaid/Pages/defau...,https://ssb.aamu.edu/PROD/bwskalog.P_DispLoginNon,www2.aamu.edu/scripts/netpricecalc/npcalc.htm,,www.aamu.edu/administrativeoffices/irpsp/insti...,www.aamu.edu/administrativeoffices/VADS/Pages/...,1,1,1,9,1,1,12,1,1,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,18,16,18,10,4,13,18,16,1,3,2,-2,-2,26620,1,290,-2,1089,Madison County,105,-86.568502,34.783368,122,1
1,100663,University of Alabama at Birmingham,,Administration Bldg Suite 1070,Birmingham,AL,35294-0110,1,5,Ray L. Watts,President,2059344011,636005396,063690705,00105200,1,www.uab.edu,www.uab.edu/students/undergraduate-admissions,www.uab.edu/students/paying-for-college,https://idm.uab.edu/myuab/login?from=ugadmapp,uab.studentaidcalculator.com/survey.aspx,www.uab.edu/students/veterans,www.uab.edu/registrar/students,www.uab.edu/students/disability/,1,1,1,9,1,1,11,1,2,1,1,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,15,14,17,9,5,15,15,15,2,5,1,The University of Alabama System,101050,13820,1,142,-2,1073,Jefferson County,107,-86.799345,33.505697,109,1
2,100690,Amridge University,Southern Christian University |Regions University,1200 Taylor Rd,Montgomery,AL,36117-3553,1,5,Michael C.Turner,President,33438738777528,237034324,126307792,02503400,1,www.amridgeuniversity.edu,www.amridgeuniversity.edu/admissions/,www.amridgeuniversity.edu/financialaid/,https://www2.amridgeuniversity.edu/Amridge/Log...,www2.amridgeuniversity.edu:9091/,www.amridgeuniversity.edu/admissions/military/,,www.amridgeuniversity.edu/pdf/Amridge%20Univer...,2,1,2,9,1,1,12,1,2,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,20,19,13,5,5,6,21,51,2,1,2,-2,-2,33860,1,-2,-2,1101,Montgomery County,102,-86.174010,32.362609,141,2
3,100706,University of Alabama in Huntsville,UAH |University of Alabama Huntsville,301 Sparkman Dr,Huntsville,AL,35899,1,5,Robert A. Altenkirch,President,2568246120,630520830,949687123,00105500,1,www.uah.edu,https://www.uah.edu/admissions,finaid.uah.edu/,register.uah.edu,finaid.uah.edu/,www.uah.edu/admissions/graduate/financial-aid/...,www.uah.edu/heoa,www.uah.edu/health-and-wellness/disability-sup...,1,1,1,9,1,1,11,1,2,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,16,17,17,9,4,12,15,16,2,3,1,The University of Alabama System,101050,26620,1,290,-2,1089,Madison County,105,-86.640449,34.724557,112,2
4,100724,Alabama State University,,915 S Jackson Street,Montgomery,AL,36104-0271,1,5,Quinton T. Ross,President,3342294100,636001101,040672685,00100500,1,www.alasu.edu,www.alasu.edu/admissions/index.aspx,www.alasu.edu/cost-aid/index.aspx,www.alasu.edu/admissions/undergrad-admissions/...,www.alasu.edu/cost-aid/forms/calculator/index....,,www.alasu.edu/search-results/index.aspx,www.alasu.edu/about-asu/the-campus/disability-...,1,1,1,9,1,1,11,1,1,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,19,13,13,10,3,13,18,21,2,2,2,-2,-2,33860,1,-2,-2,1101,Montgomery County,107,-86.295677,32.364317,131,1


In [27]:
# import admissions data
admissions_data = os.path.join(data_dir_raw, "adm2017.csv")
df_admissions = pd.read_csv(admissions_data, encoding = "ISO-8859-1")  # encoding required, "utf-8" encoding fails
df_admissions = df_admissions.rename(columns=lambda x: x.strip()) # strip whitespace, to avoid column name mismatches
df_admissions.head(5)

,UNITID,ADMCON1,ADMCON2,ADMCON3,ADMCON4,ADMCON5,ADMCON6,ADMCON7,ADMCON8,ADMCON9,XAPPLCN,APPLCN,XAPPLCNM,APPLCNM,XAPPLCNW,APPLCNW,XADMSSN,ADMSSN,XADMSSNM,ADMSSNM,XADMSSNW,ADMSSNW,XENRLT,ENRLT,XENRLM,ENRLM,XENRLW,ENRLW,XENRLFT,ENRLFT,XENRLFTM,ENRLFTM,XENRLFTW,ENRLFTW,XENRLPT,ENRLPT,XENRLPTM,ENRLPTM,XENRLPTW,ENRLPTW,XSATNUM,SATNUM,XSATPCT,SATPCT,XACTNUM,ACTNUM,XACTPCT,ACTPCT,XSATVR25,SATVR25,XSATVR75,SATVR75,XSATMT25,SATMT25,XSATMT75,SATMT75,XACTCM25,ACTCM25,XACTCM75,ACTCM75,XACTEN25,ACTEN25,XACTEN75,ACTEN75,XACTMT25,ACTMT25,XACTMT75,ACTMT75
0,100654,1,2,1,2,3,2,1,1,3,R,8610,R,3066.0,R,5544.0,R,7772.0,R,2704.0,R,5068.0,R,1294.0,R,518.0,R,776.0,R,1288.0,R,515.0,R,773.0,R,6.0,R,3.0,R,3.0,R,21.0,R,1.0,R,1270.0,R,88.0,R,365.0,R,485.0,R,360.0,R,495.0,R,16.0,R,19.0,R,14.0,R,20.0,R,15.0,R,18.0
1,100663,1,3,1,1,3,3,1,3,3,R,7555,R,2729.0,R,4826.0,R,6936.0,R,2487.0,R,4449.0,R,2299.0,R,861.0,R,1438.0,R,2228.0,R,833.0,R,1395.0,R,71.0,R,28.0,R,43.0,R,15.0,R,1.0,R,2093.0,R,94.0,R,440.0,R,630.0,R,550.0,R,740.0,R,21.0,R,28.0,R,22.0,R,31.0,R,21.0,R,26.0
2,100706,1,2,1,1,3,2,1,1,3,R,4454,R,2466.0,R,1988.0,R,3618.0,R,2082.0,R,1536.0,R,1352.0,R,867.0,R,485.0,R,1341.0,R,860.0,R,481.0,R,11.0,R,7.0,R,4.0,R,29.0,R,2.0,R,1292.0,R,96.0,R,550.0,R,660.0,R,530.0,R,670.0,R,25.0,R,31.0,R,25.0,R,33.0,R,24.0,R,29.0
3,100724,1,3,2,3,3,3,1,1,3,R,6842,R,2136.0,R,4515.0,R,6696.0,R,2071.0,R,4438.0,R,967.0,R,357.0,R,610.0,R,951.0,R,349.0,R,602.0,R,16.0,R,8.0,R,8.0,R,173.0,R,18.0,R,826.0,R,85.0,R,380.0,R,485.0,R,375.0,R,481.0,R,16.0,R,20.0,R,15.0,R,19.0,R,14.0,R,19.0
4,100751,1,2,1,1,3,3,1,1,3,R,38129,R,14705.0,R,23424.0,R,20321.0,R,7992.0,R,12329.0,R,7407.0,R,3167.0,R,4240.0,R,7385.0,R,3157.0,R,4228.0,R,22.0,R,10.0,R,12.0,R,1397.0,R,19.0,R,5981.0,R,81.0,R,530.0,R,640.0,R,520.0,R,640.0,R,23.0,R,32.0,R,23.0,R,33.0,R,21.0,R,29.0


In [28]:
# admissions data - mapping key for column values to real words
# first manually create json with definitions from `adm2017.xlsx` 
# using tabs:  `varlist`, `Description` and `imputation values`
# import json as python dict
import json

with open(os.path.join(data_dir_raw, "adm2017_key.json")) as jsonfile:
    admissions_key = json.load(jsonfile)

admissions_key

{'UNITID': 'Unique identification number of the institution',
 'ADMCON1': {'definition': 'Secondary school GPA',
  'shortname': 'admit_GPA',
  'response_type': 'int',
  'values': {'1': 'required',
   '2': 'recommended',
   '3': 'neither required nor recommended',
   '4': 'unknown'}},
 'ADMCON2': {'definition': 'Secondary school rank',
  'shortname': 'admit_class_rank',
  'response_type': 'int',
  'values': {'1': 'required',
   '2': 'recommended',
   '3': 'neither required nor recommended',
   '4': 'unknown'}},
 'ADMCON3': {'definition': 'Secondary school record',
  'shortname': 'admit_record',
  'response_type': 'int',
  'values': {'1': 'required',
   '2': 'recommended',
   '3': 'neither required nor recommended',
   '4': 'unknown'}},
 'ADMCON4': {'definition': 'Completion of college-preparatory program',
  'shortname': 'admit_college_prep',
  'response_type': 'int',
  'values': {'1': 'required',
   '2': 'recommended',
   '3': 'neither required nor recommended',
   '4': 'unknown'}},
 '

In [29]:
# work on replacing column values
df_admissions_v2 = df_admissions.copy()
df_admissions_v2.head(5)

,UNITID,ADMCON1,ADMCON2,ADMCON3,ADMCON4,ADMCON5,ADMCON6,ADMCON7,ADMCON8,ADMCON9,XAPPLCN,APPLCN,XAPPLCNM,APPLCNM,XAPPLCNW,APPLCNW,XADMSSN,ADMSSN,XADMSSNM,ADMSSNM,XADMSSNW,ADMSSNW,XENRLT,ENRLT,XENRLM,ENRLM,XENRLW,ENRLW,XENRLFT,ENRLFT,XENRLFTM,ENRLFTM,XENRLFTW,ENRLFTW,XENRLPT,ENRLPT,XENRLPTM,ENRLPTM,XENRLPTW,ENRLPTW,XSATNUM,SATNUM,XSATPCT,SATPCT,XACTNUM,ACTNUM,XACTPCT,ACTPCT,XSATVR25,SATVR25,XSATVR75,SATVR75,XSATMT25,SATMT25,XSATMT75,SATMT75,XACTCM25,ACTCM25,XACTCM75,ACTCM75,XACTEN25,ACTEN25,XACTEN75,ACTEN75,XACTMT25,ACTMT25,XACTMT75,ACTMT75
0,100654,1,2,1,2,3,2,1,1,3,R,8610,R,3066.0,R,5544.0,R,7772.0,R,2704.0,R,5068.0,R,1294.0,R,518.0,R,776.0,R,1288.0,R,515.0,R,773.0,R,6.0,R,3.0,R,3.0,R,21.0,R,1.0,R,1270.0,R,88.0,R,365.0,R,485.0,R,360.0,R,495.0,R,16.0,R,19.0,R,14.0,R,20.0,R,15.0,R,18.0
1,100663,1,3,1,1,3,3,1,3,3,R,7555,R,2729.0,R,4826.0,R,6936.0,R,2487.0,R,4449.0,R,2299.0,R,861.0,R,1438.0,R,2228.0,R,833.0,R,1395.0,R,71.0,R,28.0,R,43.0,R,15.0,R,1.0,R,2093.0,R,94.0,R,440.0,R,630.0,R,550.0,R,740.0,R,21.0,R,28.0,R,22.0,R,31.0,R,21.0,R,26.0
2,100706,1,2,1,1,3,2,1,1,3,R,4454,R,2466.0,R,1988.0,R,3618.0,R,2082.0,R,1536.0,R,1352.0,R,867.0,R,485.0,R,1341.0,R,860.0,R,481.0,R,11.0,R,7.0,R,4.0,R,29.0,R,2.0,R,1292.0,R,96.0,R,550.0,R,660.0,R,530.0,R,670.0,R,25.0,R,31.0,R,25.0,R,33.0,R,24.0,R,29.0
3,100724,1,3,2,3,3,3,1,1,3,R,6842,R,2136.0,R,4515.0,R,6696.0,R,2071.0,R,4438.0,R,967.0,R,357.0,R,610.0,R,951.0,R,349.0,R,602.0,R,16.0,R,8.0,R,8.0,R,173.0,R,18.0,R,826.0,R,85.0,R,380.0,R,485.0,R,375.0,R,481.0,R,16.0,R,20.0,R,15.0,R,19.0,R,14.0,R,19.0
4,100751,1,2,1,1,3,3,1,1,3,R,38129,R,14705.0,R,23424.0,R,20321.0,R,7992.0,R,12329.0,R,7407.0,R,3167.0,R,4240.0,R,7385.0,R,3157.0,R,4228.0,R,22.0,R,10.0,R,12.0,R,1397.0,R,19.0,R,5981.0,R,81.0,R,530.0,R,640.0,R,520.0,R,640.0,R,23.0,R,32.0,R,23.0,R,33.0,R,21.0,R,29.0


In [30]:
columns = df_admissions.columns


In [31]:
df_admissions_v2.replace(admissions_key)
df_admissions_v2.head(5)

TypeError: If a nested mapping is passed, all values of the top level mapping must be mappings

In [32]:
# df['col1'].map(di).fillna(df['col1'])

for column in columns:
    df_admissions_v2[column].replace(admissions_key[column]).fillna(df_admissions_v2[column])

df_admissions_v2.head(5)

TypeError: If a nested mapping is passed, all values of the top level mapping must be mappings

In [33]:
display(df_admissions_v2.iloc[0, 0])

100654

In [34]:
display(df_admissions_v2.iloc[0, 1])

1

In [35]:
json_keys = [key for key in admissions_key]
json_keys

['UNITID',
 'ADMCON1',
 'ADMCON2',
 'ADMCON3',
 'ADMCON4',
 'ADMCON5',
 'ADMCON6',
 'ADMCON7',
 'ADMCON8',
 'ADMCON9',
 'APPLCN',
 'APPLCNM',
 'APPLCNW',
 'ADMSSN',
 'ADMSSNM',
 'ADMSSNW',
 'ENRLT',
 'ENRLM',
 'ENRLW',
 'ENRLFT',
 'ENRLFTM',
 'ENRLFTW',
 'ENRLPT',
 'ENRLPTM',
 'ENRLPTW',
 'SATNUM',
 'SATPCT',
 'ACTNUM',
 'ACTPCT',
 'SATVR25',
 'SATVR75',
 'SATMT25',
 'SATMT75',
 'ACTCM25',
 'ACTCM75',
 'ACTEN25',
 'ACTEN75',
 'ACTMT25',
 'ACTMT75',
 'imputation_values',
 'imputation_values_usage']

In [37]:
columns

Index(['UNITID', 'ADMCON1', 'ADMCON2', 'ADMCON3', 'ADMCON4', 'ADMCON5',
       'ADMCON6', 'ADMCON7', 'ADMCON8', 'ADMCON9', 'XAPPLCN', 'APPLCN',
       'XAPPLCNM', 'APPLCNM', 'XAPPLCNW', 'APPLCNW', 'XADMSSN', 'ADMSSN',
       'XADMSSNM', 'ADMSSNM', 'XADMSSNW', 'ADMSSNW', 'XENRLT', 'ENRLT',
       'XENRLM', 'ENRLM', 'XENRLW', 'ENRLW', 'XENRLFT', 'ENRLFT', 'XENRLFTM',
       'ENRLFTM', 'XENRLFTW', 'ENRLFTW', 'XENRLPT', 'ENRLPT', 'XENRLPTM',
       'ENRLPTM', 'XENRLPTW', 'ENRLPTW', 'XSATNUM', 'SATNUM', 'XSATPCT',
       'SATPCT', 'XACTNUM', 'ACTNUM', 'XACTPCT', 'ACTPCT', 'XSATVR25',
       'SATVR25', 'XSATVR75', 'SATVR75', 'XSATMT25', 'SATMT25', 'XSATMT75',
       'SATMT75', 'XACTCM25', 'ACTCM25', 'XACTCM75', 'ACTCM75', 'XACTEN25',
       'ACTEN25', 'XACTEN75', 'ACTEN75', 'XACTMT25', 'ACTMT25', 'XACTMT75',
       'ACTMT75'],
      dtype='object')

In [36]:
missing_keys = [key for key in columns if key not in json_keys]
missing_keys

['XAPPLCN',
 'XAPPLCNM',
 'XAPPLCNW',
 'XADMSSN',
 'XADMSSNM',
 'XADMSSNW',
 'XENRLT',
 'XENRLM',
 'XENRLW',
 'XENRLFT',
 'XENRLFTM',
 'XENRLFTW',
 'XENRLPT',
 'XENRLPTM',
 'XENRLPTW',
 'XSATNUM',
 'XSATPCT',
 'XACTNUM',
 'XACTPCT',
 'XSATVR25',
 'XSATVR75',
 'XSATMT25',
 'XSATMT75',
 'XACTCM25',
 'XACTCM75',
 'XACTEN25',
 'XACTEN75',
 'XACTMT25',
 'XACTMT75']

In [44]:
for column in missing_keys:
    print(column, " : ", sorted(df_admissions_v2[column].unique()))

XAPPLCN  :  ['N', 'R']
XAPPLCNM  :  ['B', 'N', 'R', 'Z']
XAPPLCNW  :  ['B', 'N', 'R', 'Z']
XADMSSN  :  ['A', 'N', 'R']
XADMSSNM  :  ['A', 'B', 'N', 'R']
XADMSSNW  :  ['A', 'B', 'N', 'R']
XENRLT  :  ['A', 'N', 'R']
XENRLM  :  ['A', 'N', 'R']
XENRLW  :  ['A', 'N', 'R']
XENRLFT  :  ['A', 'N', 'R']
XENRLFTM  :  ['A', 'N', 'R']
XENRLFTW  :  ['A', 'N', 'R']
XENRLPT  :  ['A', 'R', 'Z']
XENRLPTM  :  ['A', 'R', 'Z']
XENRLPTW  :  ['A', 'R', 'Z']
XSATNUM  :  ['A', 'N', 'R', 'Z']
XSATPCT  :  ['A', 'B', 'N', 'R', 'Z']
XACTNUM  :  ['A', 'N', 'R', 'Z']
XACTPCT  :  ['A', 'N', 'R', 'Z']
XSATVR25  :  ['A', 'B', 'N', 'R']
XSATVR75  :  ['A', 'B', 'N', 'R']
XSATMT25  :  ['A', 'B', 'N', 'R']
XSATMT75  :  ['A', 'B', 'N', 'R']
XACTCM25  :  ['A', 'N', 'R']
XACTCM75  :  ['A', 'N', 'R']
XACTEN25  :  ['A', 'N', 'R']
XACTEN75  :  ['A', 'N', 'R']
XACTMT25  :  ['A', 'N', 'R']
XACTMT75  :  ['A', 'N', 'R']


In [ ]:
jsondict = {}
tempdict = {}

for column in missing_keys:
    missing_column_values = sorted(df_admissions_v2[column].unique())
    print(column, " : ", missing_column_values)
    tempdict = {column:{}}